# Data Parsing (Summaries)

In [2]:
# standard imports, add as needed
import pandas as pd
import numpy as np
import os
import re

In [3]:
# Gets file directory list, removes all duplicates in directory
filedir = os.listdir('Data')
filedir = [file for file in filedir if ('(1)' not in file) ]

In [4]:
r = re.compile("2017-Tulare-Woodlake-S-All\.txt")
newlist = list(filter(r.match, filedir)) # Read Note
string = ''
fp = os.path.join('Data', newlist[0])
with open(fp, 'r') as file:
    for line in file:
        string +=  line

In [5]:
pattern_1 = re.compile('(?i)(^(?=.*\b(IMPARTIAL|COUNTY|COUNSEL))(?=(.)*\b(MEASURE|ANALYSIS|COUNSEL)\b).*)')
pattern_2 = re.compile('s\/( )+\w+( )+\w+')
m1 = re.search(pattern_1, string)
m2 = re.search(pattern_2, string)

In [7]:
m1

In [9]:
print(

﻿Voter's Pamphlet Information Section

The following pages contain voter information
applicable to your ballot which may include any
or all of the following:

•	Candidate's Statements

•	Ballot Measures

•	Impartial Analyses

•	Arguments For & Against

•	Text of Measure(s)

This pamphlet section may not contain a complete list of candidates. A complete list of candidates
appears on the Sample Ballot. Each candidate's statement in this pamphlet is volunteered by the
candidate and is printed at the expense of the candidate unless otherwise determined by the
governing body.

Arguments in support of or in opposition to the proposed laws are opinions of the authors.

The text, grammar and spelling are as submitted by the authors.__

Section Informativa del Folleto de Votantes

Las siguientes paginas contienen information para el
votante relacionada con su boleta, como por ejemplo:

•	Declaraciones de los Candidatos

•	Medidas de la Boleta

•	Analisis Imparciales

•	Argumentos a Favor y en C